# Packages & Helper Functions

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, KBinsDiscretizer, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer

# show all cols in df
pd.set_option('display.max_columns', None)

# Function to do all the preprocessing

def preprocess_train_test(train_df, test_df):
    def preprocess_single(df):
        df_copy = df.copy()

        # Ensure 'trans_date_trans_time' is in datetime format
        df_copy['trans_date_trans_time'] = pd.to_datetime(df_copy['trans_date_trans_time'])
        df_copy['unix_time'] = pd.to_datetime(df_copy['unix_time'], unit='s')

        # Get month of year, day of week, and hour of day for transactions
        df_copy['trans_day_of_week'] = df_copy['trans_date_trans_time'].dt.day_name()
        df_copy['trans_month'] = df_copy['trans_date_trans_time'].dt.month_name()

        # Create 'time_of_day' column
        df_copy['time_of_day'] = (df_copy['trans_date_trans_time'].dt.hour * 60 + df_copy['trans_date_trans_time'].dt.minute) / (24 * 60)

        # Calculate age in years
        df_copy['dob'] = pd.to_datetime(df_copy['dob'])
        df_copy['age'] = df_copy['trans_date_trans_time'].dt.year - df_copy['dob'].dt.year - (
            (df_copy['trans_date_trans_time'].dt.month < df_copy['dob'].dt.month) |
            ((df_copy['trans_date_trans_time'].dt.month == df_copy['dob'].dt.month) & (df_copy['trans_date_trans_time'].dt.day < df_copy['dob'].dt.day))
        )

        # Sort the DataFrame by 'cc_num' and 'trans_date_trans_time'
        df_copy.sort_values(by=['cc_num', 'trans_date_trans_time'], inplace=True)

        # Group by 'cc_num' and calculate the time difference
        df_copy['last_purchased'] = df_copy.groupby('cc_num')['trans_date_trans_time'].diff()

        # For the first transaction for each 'cc_num', set 'last_purchased' to NaT
        df_copy['last_purchased'] = df_copy['last_purchased'].fillna(pd.NaT)

        # Convert 'last_purchased' to numerical format (total number of seconds)
        df_copy['last_purchased_secs'] = df_copy['last_purchased'].dt.total_seconds()

        # Define the Haversine distance function
        def haversine(lat1, lon1, lat2, lon2):
            R = 6371  # Radius of the Earth in kilometers
            lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
            dlat = lat2 - lat1
            dlon = lon2 - lon1
            a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
            c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
            return R * c

        # Calculate the distance to the last purchase
        df_copy['last_merch_lat'] = df_copy.groupby('cc_num')['merch_lat'].shift()
        df_copy['last_merch_long'] = df_copy.groupby('cc_num')['merch_long'].shift()

        df_copy['distance_last_purchase'] = haversine(df_copy['merch_lat'], df_copy['merch_long'], df_copy['last_merch_lat'], df_copy['last_merch_long'])

        # For the first transaction for each 'cc_num', set 'distance_last_purchase' to NaN
        df_copy['distance_last_purchase'] = df_copy['distance_last_purchase'].fillna(np.nan)

        # Create 'same_merchant_with_last_purchase' column
        df_copy['last_merchant'] = df_copy.groupby('cc_num')['merchant'].shift()
        df_copy['same_merchant_with_last_purchase'] = (df_copy['merchant'] == df_copy['last_merchant'])

        # Drop the temporary columns
        df_copy.drop(columns=['last_merch_lat', 'last_merch_long', 'last_merchant'], inplace=True)

        # Drop the specified columns
        df_copy.drop(columns=['last_purchased', 'city', 'street', 'last', 'first', 'job', 'cc_num', 'merchant', 'state', 'zip', 'lat', 'long', 'merch_lat', 'merch_long', 'trans_num', 'unix_time', 'dob'], inplace=True)

        # Sort the DataFrame by the original index to maintain the original order
        df_copy.sort_index(inplace=True)

        return df_copy

    train_df_preprocessed = preprocess_single(train_df)
    test_df_preprocessed = preprocess_single(test_df)

    return train_df_preprocessed, test_df_preprocessed

# Takes in processsed train and test data, removes all January 2019 data, and returns train, val, test
def process_fraud_data(train_df_preprocessed, test_df_preprocessed):
    # Combine the preprocessed data
    combined_df = pd.concat([train_df_preprocessed, test_df_preprocessed])

    # Remove rows where 'last_purchased_secs' is NaN
    combined_df = combined_df.dropna(subset=['last_purchased_secs'])

    # Remove all January 2019 data
    combined_df = combined_df[~((combined_df['trans_date_trans_time'].dt.year == 2019) & (combined_df['trans_date_trans_time'].dt.month == 1))]

    # Drop trans_date_trans_time
    combined_df = combined_df.drop(columns=['trans_date_trans_time'])

    # Split the combined data into train, validation, and test sets by order
    total_samples = len(combined_df)
    train_end = int(0.64 * total_samples)
    val_end = int(0.80 * total_samples)

    train_data = combined_df.iloc[:train_end]
    val_data = combined_df.iloc[train_end:val_end]
    test_data = combined_df.iloc[val_end:]

    return train_data, val_data, test_data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Load & Preprocess Data

In [ ]:
train_df = pd.read_csv("/content/drive/Shared drives/Machine Learning/Project/fraudTrain.csv", index_col=0)
test_df = pd.read_csv("/content/drive/Shared drives/Machine Learning/Project/fraudTest.csv", index_col=0)
train_preprocessed, test_preprocessed = preprocess_train_test(train_df, test_df)
train, val, test = process_fraud_data(train_preprocessed, train_preprocessed)
train.head()

,category,amt,gender,city_pop,is_fraud,trans_day_of_week,trans_month,time_of_day,age,last_purchased_secs,distance_last_purchase,same_merchant_with_last_purchase
52525,grocery_net,19.46,F,1423,0,Friday,February,0.001389,20,16027.0,155.644412,False
52526,misc_net,13.01,M,471,0,Friday,February,0.002083,51,163101.0,121.187723,False
52527,gas_transport,50.02,M,471,0,Friday,February,0.003472,77,55381.0,28.230127,False
52528,entertainment,6.11,M,192805,0,Friday,February,0.004167,55,91351.0,132.898691,False
52529,grocery_net,32.14,M,18408,0,Friday,February,0.005556,46,971.0,81.694148,False


In [ ]:
categorical_features = ['category', 'gender', 'trans_day_of_week', 'trans_month', 'same_merchant_with_last_purchase']
continuous_features = ['amt', "city_pop", "time_of_day", "age", 'last_purchased_secs', 'distance_last_purchase']
train = pd.get_dummies(train, columns=categorical_features, drop_first=True)

# Engineeri val & test in the same way as train set, if the one-hot encoding doesn't exist in test set, fill False

val = pd.get_dummies(val, columns=categorical_features, drop_first=True)
val = val.reindex(columns=train.columns, fill_value=0)
test = pd.get_dummies(test, columns=categorical_features, drop_first=True)
test = test.reindex(columns=train.columns, fill_value=0)
train.head()

,amt,city_pop,is_fraud,time_of_day,age,last_purchased_secs,distance_last_purchase,category_food_dining,category_gas_transport,category_grocery_net,category_grocery_pos,category_health_fitness,category_home,category_kids_pets,category_misc_net,category_misc_pos,category_personal_care,category_shopping_net,category_shopping_pos,category_travel,gender_M,trans_day_of_week_Monday,trans_day_of_week_Saturday,trans_day_of_week_Sunday,trans_day_of_week_Thursday,trans_day_of_week_Tuesday,trans_day_of_week_Wednesday,trans_month_August,trans_month_December,trans_month_February,trans_month_January,trans_month_July,trans_month_June,trans_month_March,trans_month_May,trans_month_November,trans_month_October,trans_month_September,same_merchant_with_last_purchase_True
52525,19.46,1423,0,0.001389,20,16027.0,155.644412,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False
52526,13.01,471,0,0.002083,51,163101.0,121.187723,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False
52527,50.02,471,0,0.003472,77,55381.0,28.230127,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False
52528,6.11,192805,0,0.004167,55,91351.0,132.898691,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False
52529,32.14,18408,0,0.005556,46,971.0,81.694148,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False


In [ ]:
val.head()

,amt,city_pop,is_fraud,time_of_day,age,last_purchased_secs,distance_last_purchase,category_food_dining,category_gas_transport,category_grocery_net,category_grocery_pos,category_health_fitness,category_home,category_kids_pets,category_misc_net,category_misc_pos,category_personal_care,category_shopping_net,category_shopping_pos,category_travel,gender_M,trans_day_of_week_Monday,trans_day_of_week_Saturday,trans_day_of_week_Sunday,trans_day_of_week_Thursday,trans_day_of_week_Tuesday,trans_day_of_week_Wednesday,trans_month_August,trans_month_December,trans_month_February,trans_month_January,trans_month_July,trans_month_June,trans_month_March,trans_month_May,trans_month_November,trans_month_October,trans_month_September,same_merchant_with_last_purchase_True
400890,54.99,836,0,0.298611,37,4012.0,138.996144,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,0,False,0,0,True,0,0,0,False,False,False,False
400891,56.25,73,0,0.298611,84,21891.0,50.205417,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,0,False,0,0,True,0,0,0,False,False,False,False
400892,39.94,22305,0,0.299306,80,12629.0,65.398824,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,0,False,0,0,True,0,0,0,False,False,False,False
400893,53.38,2906700,0,0.299306,34,2601.0,12.922509,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,0,False,0,0,True,0,0,0,False,False,False,False
400894,3.54,23045,0,0.300000,60,11273.0,102.414743,False,False,False,False,False,False,False,False,False,False,True,False,False,True,True,False,False,False,False,False,0,False,0,0,True,0,0,0,False,False,False,False


In [ ]:
test.head()

,amt,city_pop,is_fraud,time_of_day,age,last_purchased_secs,distance_last_purchase,category_food_dining,category_gas_transport,category_grocery_net,category_grocery_pos,category_health_fitness,category_home,category_kids_pets,category_misc_net,category_misc_pos,category_personal_care,category_shopping_net,category_shopping_pos,category_travel,gender_M,trans_day_of_week_Monday,trans_day_of_week_Saturday,trans_day_of_week_Sunday,trans_day_of_week_Thursday,trans_day_of_week_Tuesday,trans_day_of_week_Wednesday,trans_month_August,trans_month_December,trans_month_February,trans_month_January,trans_month_July,trans_month_June,trans_month_March,trans_month_May,trans_month_November,trans_month_October,trans_month_September,same_merchant_with_last_purchase_True
799013,50.56,737,0,0.947222,32,20638.0,94.540838,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,True,False,0,True,False,False,0,False,False,False,0,0,0,False
799014,11.09,139,0,0.947917,65,18859.0,82.891431,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,0,True,False,False,0,False,False,False,0,0,0,False
799015,41.20,1120,0,0.948611,62,159665.0,32.075905,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,True,False,0,True,False,False,0,False,False,False,0,0,0,False
799016,8.65,13602,0,0.948611,35,8883.0,120.786297,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,True,False,0,True,False,False,0,False,False,False,0,0,0,False
799017,59.24,18128,0,0.949306,29,4263.0,88.726573,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,0,True,False,False,0,False,False,False,0,0,0,False


In [ ]:
x_train = train.drop(columns=['is_fraud'])
y_train = train['is_fraud']
x_val = val.drop(columns=['is_fraud'])
y_val = val['is_fraud']

# Balance Distribution

## EDA

In [ ]:
y_train.value_counts()

,count
is_fraud,
0,1583366
1,9056


## Random Over-Sampling

In [ ]:
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import classification_report, roc_auc_score
import xgboost as xgb

# Define the desired ratio for oversampling
sampling_strategy = {0: y_train.value_counts()[0], 1: y_train.value_counts()[0] * 10}

# Initialize and apply random oversampling
oversampler = RandomOverSampler(sampling_strategy=sampling_strategy, random_state=42)
x_train_resampled, y_train_resampled = oversampler.fit_resample(x_train, y_train)

# Verify class distribution after oversampling
print(y_train_resampled.value_counts())

# Initialize and train the XGBoost model with GPU support
model = xgb.XGBClassifier(random_state=42, tree_method='gpu_hist', gpu_id=0) # Use gpu_hist for GPU acceleration
model.fit(x_train_resampled, y_train_resampled)

# Make predictions on the validation set
y_pred_val = model.predict(x_val)
y_prob_val = model.predict_proba(x_val)[:, 1]

# Evaluate the model
print(classification_report(y_val, y_pred_val))
print("ROC AUC Score:", roc_auc_score(y_val, y_prob_val))


is_fraud
1    15833660
0     1583366
Name: count, dtype: int64


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [18:40:03] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [18:40:21] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


              precision    recall  f1-score   support

           0       1.00      0.99      1.00    396104
           1       0.38      1.00      0.55      2002

    accuracy                           0.99    398106
   macro avg       0.69      1.00      0.77    398106
weighted avg       1.00      0.99      0.99    398106

ROC AUC Score: 0.9999171015349847


## Random Down-sampling

In [ ]:
# prompt: random down sampling
from imblearn.under_sampling import RandomUnderSampler
# Define the desired ratio for undersampling
sampling_ratio = 0.1 # 1:10 ratio
# Create a RandomUnderSampler object with the specified sampling strategy
rus = RandomUnderSampler(sampling_strategy=sampling_ratio, random_state=42)
# Apply random undersampling to the training data
x_train_undersample, y_train_undersample = rus.fit_resample(x_train, y_train)
# ### XgBoost (using undersampled data)
# Initialize and train the XGBoost classifier with GPU support
xgb_model_undersampled = xgb.XGBClassifier(random_state=42, tree_method='gpu_hist', gpu_id=0) # Use gpu_hist for GPU acceleration
xgb_model_undersampled.fit(x_train_undersample, y_train_undersample)
# Make predictions on the validation set
y_pred_xgb_undersampled = xgb_model_undersampled.predict(x_val)
# Evaluate the model
print(classification_report(y_val, y_pred_xgb_undersampled))


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [18:40:22] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [18:40:23] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


              precision    recall  f1-score   support

           0       1.00      1.00      1.00    396104
           1       0.69      0.98      0.81      2002

    accuracy                           1.00    398106
   macro avg       0.85      0.99      0.90    398106
weighted avg       1.00      1.00      1.00    398106



## SMOTE Over-sampling

In [ ]:
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier

# Define the desired ratio for SMOTE
sampling_ratio = 0.1  # 1:10 ratio

# Create a SMOTE object with the specified sampling strategy
smote = SMOTE(sampling_strategy=sampling_ratio, random_state=42)

# Apply SMOTE to the training data
x_train_smote, y_train_smote = smote.fit_resample(x_train, y_train)

# ### XgBoost (using SMOTE data)
# Initialize and train the XGBoost classifier with GPU support
xgb_model_smote = XGBClassifier(random_state=42, tree_method='gpu_hist', gpu_id=0) # Use gpu_hist for GPU acceleration
xgb_model_smote.fit(x_train_smote, y_train_smote)

# Make predictions on the validation set
y_pred_xgb_smote = xgb_model_smote.predict(x_val)

# Evaluate the model
print(classification_report(y_val, y_pred_xgb_smote))

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [22:58:19] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [22:58:21] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


              precision    recall  f1-score   support

           0       1.00      1.00      1.00    396104
           1       0.93      0.87      0.90      2002

    accuracy                           1.00    398106
   macro avg       0.96      0.93      0.95    398106
weighted avg       1.00      1.00      1.00    398106



SMOTE performs the best.

# Final Model Selection

In [ ]:
stratified_kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
x_train_smote, y_train_smote = smote.fit_resample(x_train, y_train)

## Logistics

In [ ]:
import statsmodels.api as sm
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, roc_auc_score

# Add an intercept term for statsmodels
x_train_smote_const = sm.add_constant(x_train_smote)
x_val_const = sm.add_constant(x_val)


# Fit the logistic regression model using statsmodels
logit_model = sm.Logit(np.asarray(y_train_smote), x_train_smote_const.astype(float)).fit()
print(logit_model.summary())

Optimization terminated successfully.
         Current function value: 0.075174
         Iterations 10
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:              1741702
Model:                          Logit   Df Residuals:                  1741663
Method:                           MLE   Df Model:                           38
Date:                Sun, 09 Mar 2025   Pseudo R-squ.:                  0.7532
Time:                        18:44:54   Log-Likelihood:            -1.3093e+05
converged:                       True   LL-Null:                   -5.3058e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                            coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------
const                                   -16.8177     

In [ ]:
# Make predictions on the validation set
y_prob_lr = logit_model.predict(x_val_const.astype(float))  # Probability predictions
y_pred_lr = (y_prob_lr >= 0.5).astype(int)  # Convert to binary predictions

# Evaluate the model
print(classification_report(y_val, y_pred_lr))
print("ROC AUC Score for Logistic Regression:", roc_auc_score(y_val, y_prob_lr))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    396104
           1       0.41      0.28      0.33      2002

    accuracy                           0.99    398106
   macro avg       0.70      0.64      0.66    398106
weighted avg       0.99      0.99      0.99    398106

ROC AUC Score for Logistic Regression: 0.8453203835729637


In [ ]:
import pickle

# Save model
with open("/content/drive/Shared drives/Machine Learning/Project/logi_model_balanced.pkl", "wb") as f:
    pickle.dump(logit_model, f)

# # Load model
# with open("xgb_model.pkl", "rb") as f:
#     xgb_loaded = pickle.load(f)


## XgBoost

In [ ]:
# prompt: fit xgboost and perform grid search on hyper parameter on smote train set and test on validation set

# Define the parameter grid for XGBoost
param_grid_xgb = {
    'n_estimators': [100, 200],
    'max_depth': [3, 6],
    'learning_rate': [0.1, 0.3],
    'subsample': [1],
    'colsample_bytree': [1]
}

# Initialize GridSearchCV for XGBoost
grid_search_xgb = GridSearchCV(estimator=XGBClassifier(random_state=42, tree_method='gpu_hist', gpu_id=0),
                              param_grid=param_grid_xgb,
                               cv=stratified_kfold, scoring='roc_auc', n_jobs=-1)

# Fit the grid search to the SMOTE training data
grid_search_xgb.fit(x_train_smote, y_train_smote)

# Get the best parameters and best score
best_params_xgb = grid_search_xgb.best_params_
best_score_xgb = grid_search_xgb.best_score_

print(f"Best parameters for XGBoost: {best_params_xgb}")
print(f"Best ROC AUC score for XGBoost: {best_score_xgb}")

# Train an XGBoost model with the best hyperparameters
best_xgb_model = XGBClassifier(**best_params_xgb, random_state=42, tree_method='gpu_hist', gpu_id=0)
best_xgb_model.fit(x_train_smote, y_train_smote)

# Make predictions on the validation set
y_pred_xgb = best_xgb_model.predict(x_val)
y_prob_xgb = best_xgb_model.predict_proba(x_val)[:, 1]

# Evaluate the model
print(classification_report(y_val, y_pred_xgb))
print("ROC AUC Score for XGBoost:", roc_auc_score(y_val, y_prob_xgb))


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [18:05:38] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


Best parameters for XGBoost: {'colsample_bytree': 1, 'learning_rate': 0.3, 'max_depth': 6, 'n_estimators': 200, 'subsample': 1}
Best ROC AUC score for XGBoost: 0.9999475931939551


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [18:05:42] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [18:05:45] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


              precision    recall  f1-score   support

           0       1.00      1.00      1.00    396104
           1       0.97      0.90      0.94      2002

    accuracy                           1.00    398106
   macro avg       0.99      0.95      0.97    398106
weighted avg       1.00      1.00      1.00    398106

ROC AUC Score for XGBoost: 0.9996907567015417


In [ ]:
import pickle

# Save model
with open("/content/drive/Shared drives/Machine Learning/Project/xgb_model_balanced.pkl", "wb") as f:
    pickle.dump(best_xgb_model, f)

# # Load model
# with open("xgb_model.pkl", "rb") as f:
#     xgb_loaded = pickle.load(f)


## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV

# Initialize Random Forest Classifier
rf_classifier = RandomForestClassifier(random_state=42)

# Define the parameter grid for GridSearchCV
param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [None, 20],
    'min_samples_split': [2, 6],
    'min_samples_leaf': [1, 3]
}

# Initialize GridSearchCV
stratified_kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# Perform Grid Search with Stratified K-Fold CV
grid_search = GridSearchCV(estimator=rf_classifier,
              param_grid=param_grid,
              cv=stratified_kfold,  # Use stratified KFold
              scoring='roc_auc',
              n_jobs=-1,verbose=2)

# Fit the grid search to the SMOTE training data
grid_search.fit(x_train_smote, y_train_smote)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(f"Best parameters: {best_params}")
print(f"Best ROC AUC score: {best_score}")

# Train a Random Forest model with the best hyperparameters
best_rf_model = RandomForestClassifier(**best_params, random_state=42)
best_rf_model.fit(x_train_smote, y_train_smote)

# Make predictions on the validation set
y_pred_rf = best_rf_model.predict(x_val)
y_prob_rf = best_rf_model.predict_proba(x_val)[:, 1]


# Evaluate the model
print(classification_report(y_val, y_pred_rf))
print("ROC AUC Score:", roc_auc_score(y_val, y_prob_rf))

Fitting 3 folds for each of 16 candidates, totalling 48 fits
Best parameters: {'max_depth': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 100}
Best ROC AUC score: 0.9998782632132245
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    396104
           1       0.97      0.87      0.92      2002

    accuracy                           1.00    398106
   macro avg       0.99      0.93      0.96    398106
weighted avg       1.00      1.00      1.00    398106

ROC AUC Score: 0.9998816191987683


In [ ]:
import pickle

# Save model
with open("/content/drive/Shared drives/Machine Learning/Project/RF_model_balanced.pkl", "wb") as f:
    pickle.dump(best_rf_model, f)

# # Load model
# with open("xgb_model.pkl", "rb") as f:
#     xgb_loaded = pickle.load(f)


## SVM

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, roc_auc_score

# Define the parameter grid for Linear SVM
param_grid_svm = {
    'C': [0.1, 1, 10]  # Regularization parameter
}

# Initialize GridSearchCV for LinearSVC
grid_search_svm = GridSearchCV(estimator=LinearSVC(random_state=42, max_iter=10000, verbose=1),
                                param_grid=param_grid_svm,
                                cv=stratified_kfold,
                                scoring='roc_auc',
                                n_jobs=-1,
                                verbose=2)

# Fit the grid search to the SMOTE training data
grid_search_svm.fit(x_train_smote, y_train_smote)

# Get the best parameters and best score
best_params_svm = grid_search_svm.best_params_
best_score_svm = grid_search_svm.best_score_

print(f"Best parameters for Linear SVM: {best_params_svm}")
print(f"Best ROC AUC score for Linear SVM: {best_score_svm}")

# Train an SVM model with the best hyperparameters
best_svm_model = LinearSVC(**best_params_svm, random_state=42, max_iter=10000)
best_svm_model.fit(x_train_smote, y_train_smote)

# Convert LinearSVC to support probability estimation
calibrated_svm = CalibratedClassifierCV(best_svm_model)
calibrated_svm.fit(x_train_smote, y_train_smote)

# Make predictions on the validation set
y_pred_svm = calibrated_svm.predict(x_val)
y_prob_svm = calibrated_svm.predict_proba(x_val)[:, 1]

# Evaluate the model
print(classification_report(y_val, y_pred_svm))
print("ROC AUC Score for Linear SVM:", roc_auc_score(y_val, y_prob_svm))


Fitting 3 folds for each of 3 candidates, totalling 9 fits
[LibLinear]Best parameters for Linear SVM: {'C': 10}
Best ROC AUC score for Linear SVM: 0.832298639678215
              precision    recall  f1-score   support

           0       1.00      0.99      1.00    396104
           1       0.25      0.38      0.30      2002

    accuracy                           0.99    398106
   macro avg       0.62      0.69      0.65    398106
weighted avg       0.99      0.99      0.99    398106

ROC AUC Score for Linear SVM: 0.8147801091119007


In [ ]:
import pickle

# Save model
with open("/content/drive/Shared drives/Machine Learning/Project/svm_model_balanced.pkl", "wb") as f:
    pickle.dump(best_svm_model, f)

# # Load model
# with open("xgb_model.pkl", "rb") as f:
#     xgb_loaded = pickle.load(f)


# Not Balance Random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV

# Initialize Random Forest Classifier
rf_classifier = RandomForestClassifier(random_state=42)

# Define the parameter grid for GridSearchCV
param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [None, 20],
    'min_samples_split': [2, 6],
    'min_samples_leaf': [1, 3]
}

# Initialize GridSearchCV
stratified_kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# Perform Grid Search with Stratified K-Fold CV
grid_search = GridSearchCV(estimator=rf_classifier,
              param_grid=param_grid,
              cv=stratified_kfold,  # Use stratified KFold
              scoring='roc_auc',
              n_jobs=-1,verbose=2)

# Fit the grid search to the  training data
grid_search.fit(x_train, y_train)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(f"Best parameters: {best_params}")
print(f"Best ROC AUC score: {best_score}")

# Train a Random Forest model with the best hyperparameters
best_rf_model = RandomForestClassifier(**best_params, random_state=42)
best_rf_model.fit(x_train, y_train)

# Make predictions on the validation set
y_pred_rf = best_rf_model.predict(x_val)
y_prob_rf = best_rf_model.predict_proba(x_val)[:, 1]




In [ ]:
best_params

{'max_depth': 20,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 100}

In [ ]:
import pickle

# Save model
with open("/content/drive/Shared drives/Machine Learning/Project/RF_model_notbalanced.pkl", "wb") as f:
    pickle.dump(best_rf_model, f)

In [ ]:
# Evaluate the model
from sklearn.metrics import classification_report, roc_auc_score
print(classification_report(y_val, y_pred_rf))
print("ROC AUC Score:", roc_auc_score(y_val, y_prob_rf))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    396104
           1       1.00      0.85      0.92      2002

    accuracy                           1.00    398106
   macro avg       1.00      0.93      0.96    398106
weighted avg       1.00      1.00      1.00    398106

ROC AUC Score: 0.9999827692352887


# SVM Without sampling

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, roc_auc_score

# Define the parameter grid for Linear SVM
param_grid_svm = {
    'C': [0.1, 1, 10]  # Regularization parameter
}

# Initialize GridSearchCV for LinearSVC
grid_search_svm = GridSearchCV(estimator=LinearSVC(random_state=42, max_iter=10000, verbose=1),
                                param_grid=param_grid_svm,
                                cv=stratified_kfold,
                                scoring='roc_auc',
                                n_jobs=-1,
                                verbose=2)

# Fit the grid search to the training data
grid_search_svm.fit(x_train, y_train)

# Get the best parameters and best score
best_params_svm = grid_search_svm.best_params_
best_score_svm = grid_search_svm.best_score_

print(f"Best parameters for Linear SVM: {best_params_svm}")
print(f"Best ROC AUC score for Linear SVM: {best_score_svm}")

# Train an SVM model with the best hyperparameters
best_svm_model = LinearSVC(**best_params_svm, random_state=42, max_iter=10000)
best_svm_model.fit(x_train, y_train)

# Convert LinearSVC to support probability estimation
calibrated_svm = CalibratedClassifierCV(best_svm_model)
calibrated_svm.fit(x_train, y_train)

# Make predictions on the validation set
y_pred_svm = calibrated_svm.predict(x_val)
y_prob_svm = calibrated_svm.predict_proba(x_val)[:, 1]

# Evaluate the model
print(classification_report(y_val, y_pred_svm))
print("ROC AUC Score for Linear SVM:", roc_auc_score(y_val, y_prob_svm))

Fitting 3 folds for each of 3 candidates, totalling 9 fits
[LibLinear]Best parameters for Linear SVM: {'C': 1}
Best ROC AUC score for Linear SVM: 0.6070311381597322
              precision    recall  f1-score   support

           0       0.99      1.00      1.00    396104
           1       0.00      0.00      0.00      2002

    accuracy                           0.99    398106
   macro avg       0.50      0.50      0.50    398106
weighted avg       0.99      0.99      0.99    398106

ROC AUC Score for Linear SVM: 0.5699612174628838
